# 0. Imports and functions definitions

In [ ]:
from __init__ import *

### Define your variables

In [ ]:
folder    = "/pc/choozdsk01/DATA/GEANT4/megacell_v3/"
sensors   = ["Arapuca","SiPM1","SiPM2"]
my_file   = "megacell_v3_ref_3mmBiggerBox.root"
debug     = True

In [ ]:
my_data = extract_branches(folder=folder, root_file=my_file, sensors=sensors, debug=debug)

In [ ]:
filename = list(my_data.keys())[0]
data = {
  "SiPM1": my_data[filename]["SiPM1"]["AccumHits"],
  "SiPM2": my_data[filename]["SiPM2"]["AccumHits"],
  "COMBI": my_data[filename]["SiPM1"]["AccumHits"] + my_data[filename]["SiPM2"]["AccumHits"],
  "XA": my_data[filename]["Arapuca"]["AccumHits"]
}

# Save data as numpy arrays
# for key, value in data.items():
#   np.save(key, value)

xbins = 100
density = False

fig = make_subplots(rows=2, cols=1, subplot_titles=("SiPM1 + SiPM2", "Arapuca"))

for idx, (key, value) in enumerate(data.items()):  
  if idx == 3:
    bins, edges = np.histogram(value, bins=10*xbins, density=True)
    bins = bins/np.max(bins)
    fig.add_trace(go.Bar(x=edges, y=bins, name=key, opacity=0.8), row=2, col=1)
  
  else:
    bins, edges = np.histogram(value, bins=xbins, density=True)
    bins = bins/np.max(bins)
    fig.add_trace(go.Bar(x=edges, y=bins, name=key, opacity=0.8), row=1, col=1)

fig.update_layout(title_text=filename,
  barmode="overlay", bargap=0, template="presentation", height=800, width=800, showlegend=True,
  xaxis2_title_text="#Photons",
  legend_title_text="Sensor",
  legend=dict(x=.01, y=.01, xanchor="left", yanchor="bottom", orientation="v"))

fig.update_xaxes(showline=True, showgrid=True, mirror=True)
fig.update_yaxes(title_text="Norm (a.u.)", ticks="outside", showline=True, showgrid=True, mirror=True)
fig.show()
# Save figure
fig.write_image(f'../results/images/{filename.split(".root")[0]}_AccumHitsDistribution.png')